In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title='AiiDAlab QE'

In [ ]:
%%capture
from aiida import load_profile

load_profile()

In [ ]:
# Fix pybel import path
try:
    import sys

    sys.modules["pybel"] = __import__("openbabel", globals(), locals(), ["pybel"]).pybel
except Exception:
    pass

In [ ]:
import urllib.parse as urlparse

from aiidalab_qe.app.main import QeApp

url = urlparse.urlsplit(jupyter_notebook_url)  # noqa F821
query = urlparse.parse_qs(url.query)
pk = query["pk"][0] if "pk" in query else None

app = QeApp(process=pk)

In [ ]:
app.load()